# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [428]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [429]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y



In [430]:
x,y= generate_data()
print(x.shape, y.shape)

(1000, 3) (1000,)


In [431]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
  

  np.random.seed(0) 
  
  train_size = 0.8
  n = int(len(x)*train_size)
  indices = np.arange(len(x))
  np.random.shuffle(indices)
  train_idx = indices[: n]
  test_idx = indices[n:]
  X_train, y_train = x[train_idx], y[train_idx]
  X_test, y_test = x[test_idx], y[test_idx]

  return X_train, y_train, X_test, y_test
  

In [432]:
X_train, y_train, X_test, y_test= split_data(x,y, train_size= 0.8) # split your data into x_train, x_test, y_train, y_test
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [433]:
def covariance(x, mu):
  N,d = x.shape
  K= mu.shape[0]
  sigma = np.zeros((d,d))
  for k in range(K):
   for i in range(d):
     for j in range(d):
       sigma[i,j] = (1/(N-1))* np.sum((x[:,i] - mu[i]) * (x[:,j] - mu[j]))
       
  return sigma


In [434]:
covariance(x,x.mean(axis=0))

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [435]:
np.cov(x, rowvar=False)

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [445]:
class GDA:
  def __init__(self):
    self.mu=None
    self.phi = None
    self.sigma = None

  def fit(self,x,y):
    k=np.unique(y).shape[0] # Number of class.
    d= x.shape[1]# input dim
    m= x.shape[0] # Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu = np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros(d)# d-dimension

    ## START THE LEARNING: estimate mu, phi and sigma.
    for label in range(k):
      
      self.phi[label] = np.sum(label==y) / m
      self.mu[label] =  np.mean(x[label==y], axis=0)
      self.sigma[label] = covariance(x[label== y], self.mu[label])
    # return self.mu ,self.phi ,self.sigma

  def predict_proba(self,x):
    x= x.reshape(-x.shape[0],1)
    d= x.shape[1]
    k= self.mu.shape[0]
    k_class=k # Number of classes we have in our case it's k = 2
    
    ## START THE LEARNING: estimate mu, phi and sigma
    score= np.zeros((x.shape[0], self.mu.shape[0]))
    for label in range(k_class):
      
      sigma_det = np.linalg.det(self.sigma[label])**1/2
      sigma_inv = np.linalg.inv(self.sigma[label])
      d1 = (2*np.pi)**d/2
      for i in range(x.shape[0]):
        y_preds=[]
        
     
        score[i,label] = 1/(d1 * sigma_det)*(np.exp((x[i] - self.mu[label])@sigma_inv@(x[i] - self.mu[label])))*self.phi[label]
        score = np.log(score)
      
        # print(1/(d1 * sigma_det)*(np.exp((x[i] - self.mu[k]).T.reshape(d,1)@sigma_inv@(x[i] - self.mu[k])))) 
        # print(np.exp((x[i] - self.mu[label]).T@sigma_inv))
        
    return score


  def predict(self,x):
    predict = self.predict_proba(x)
    y_preds = np.argmax(predict)
    return y_preds
  
  def accuracy(self, y, ypreds):
    accuracy = (y==ypreds)*100
    return accuracy

In [446]:
model= GDA()
model.fit(X_train,y_train)

In [447]:
yproba= model.predict_proba(X_test)
yproba

TypeError: ignored

In [444]:
ypreds= model.predict(X_test)
ypreds


<ipython-input-440-181b3c2ff881>:42: RuntimeWarning: overflow encountered in exp
  score[i,label] = 1/(d1 * sigma_det)*(np.exp((x[i] - self.mu[label])@sigma_inv@(x[i] - self.mu[label])))*self.phi[label]


96

In [443]:
model.accuracy(y_test, ypreds)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])